Parul Pandey님의 https://www.kaggle.com/parulpandey/eda-and-preprocessing-for-bert    
https://www.kaggle.com/raenish/tweet-sentiment-insight-eda 에서 영감을 받았습니다

In [108]:
# 파일시스템 관련
import os

# 데이터 처리 관련
import numpy as np
import pandas as pd

# 텍스트 전처리 관련
import re
import string
import nltk
from nltk.corpus import stopwords

# 데이터시각화 관련
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# sklearn 
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer

#Transformers
# 4.4.0에서 버전문제 발생하여 3.3.1로 reinstall
from transformers import BertTokenizer
from transformers import *
import tokenizers
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

In [109]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## jaccard similarity 코드

In [110]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

### train을 train, test로 나눈다

In [111]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(train_ori,
#                                 test_size = 0.2,
#                                 random_state = 123)


In [112]:
# print(train.shape)
# print(test.shape)
# print(train.shape[0] + _test.shape[0])

In [113]:
# train.to_csv("./train_split_v1.csv")
# test.to_csv("./test_split_v1.csv")

# train_t = pd.read_csv("./train_split_v1.csv")
# train_t.head()

### hyuyoo code merge

In [114]:
train = pd.read_csv("./train_split.csv")
test = pd.read_csv("./test_split.csv")

In [115]:
test.head()

,textID,text,selected_text,sentiment
0,85a02242e3,"Its been a slow day at home, one of my kids is...","Its been a slow day at home, one of my kids is...",neutral
1,a0e0ed4311,about to head to Starbucks. was gonna take the...,missed,negative
2,f5f2a709ca,I like them. I mean I like what`s left of them.,I like them.,positive
3,064985c587,Grad present photos on facebook,Grad present photos on facebook,neutral
4,5ab37ce023,"That`s kinda cute, to be honest","kinda cute, to be honest",positive


In [116]:
print(train.sentiment.value_counts())
print()
print(train.sentiment.value_counts(normalize=True))

neutral     8919
positive    6838
negative    6227
Name: sentiment, dtype: int64

neutral     0.405704
positive    0.311044
negative    0.283251
Name: sentiment, dtype: float64


In [117]:
print(test.sentiment.value_counts())
print()
print(test.sentiment.value_counts(normalize=True))

neutral     2198
positive    1744
negative    1554
Name: sentiment, dtype: int64

neutral     0.399927
positive    0.317322
negative    0.282751
Name: sentiment, dtype: float64


-> train, test의 감정별 트윗 비율이 거의 유사하다

### test를 X, y로 분리

In [118]:
test_x = pd.DataFrame(test, columns=["textID","text","sentiment"])
test_x.head(3)

,textID,text,sentiment
0,85a02242e3,"Its been a slow day at home, one of my kids is...",neutral
1,a0e0ed4311,about to head to Starbucks. was gonna take the...,negative
2,f5f2a709ca,I like them. I mean I like what`s left of them.,positive


In [119]:
test_y = test.selected_text
test_y.head()

0    Its been a slow day at home, one of my kids is...
1                                               missed
2                                         I like them.
3                      Grad present photos on facebook
4                             kinda cute, to be honest
Name: selected_text, dtype: object

In [39]:
print(test_x.shape, test_y.shape)
print(test_x.columns, test_y.name)

(5496, 3) (5496,)
Index(['textID', 'text', 'sentiment'], dtype='object') selected_text


# 0.711~715 roBERTa model

In [40]:
# train, split -> merged from hyuyoo

In [120]:
def read_train():
    train=pd.read_csv('./train_split.csv')
    train['text']=train['text'].astype(str)
    train['selected_text']=train['selected_text'].astype(str)
    return train

def read_test():
    test=pd.read_csv('./test_split.csv')
    test['text']=test['text'].astype(str)
    return test
    
train_df = read_train()
test_df_ori = read_test()

In [121]:
tf.test.is_gpu_available()

True

In [122]:
train_df.head(2)

,textID,text,selected_text,sentiment
0,b7797a8477,I created my account just to get a chance to ...,: I ADMIRE YOU! you`re amazing! you inspire me...,positive
1,cbdb23868f,too kind Jim too kind brother,kind,positive


In [123]:
print(train_df.shape, test_df_ori.shape)

(21984, 4) (5496, 4)


In [124]:
test_df_ori.head(2)

,textID,text,selected_text,sentiment
0,85a02242e3,"Its been a slow day at home, one of my kids is...","Its been a slow day at home, one of my kids is...",neutral
1,a0e0ed4311,about to head to Starbucks. was gonna take the...,missed,negative


In [125]:
test_df = pd.DataFrame(test_df_ori, columns=["textID","text","sentiment"])
answer_st = test_df_ori["selected_text"]

In [126]:
print(test_df.columns.values, ",", answer_st.name)

['textID' 'text' 'sentiment'] , selected_text


In [127]:
test_df.head(5)

,textID,text,sentiment
0,85a02242e3,"Its been a slow day at home, one of my kids is...",neutral
1,a0e0ed4311,about to head to Starbucks. was gonna take the...,negative
2,f5f2a709ca,I like them. I mean I like what`s left of them.,positive
3,064985c587,Grad present photos on facebook,neutral
4,5ab37ce023,"That`s kinda cute, to be honest",positive


In [128]:
answer_st[:5]

0    Its been a slow day at home, one of my kids is...
1                                               missed
2                                         I like them.
3                      Grad present photos on facebook
4                             kinda cute, to be honest
Name: selected_text, dtype: object

## Data Preprocessing

In [129]:
MAX_LEN = 96
PATH = './input/tf-roberta/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}

In [51]:
ct = train_df.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train_df.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train_df.loc[k,'text'].split())
    text2 = " ".join(train_df.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train_df.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[k,:len(enc.ids)+5] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

In [52]:
ct = test_df.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test_df.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test_df.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test_df.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1

## Model

In [53]:
def scheduler(epoch):
    return 3e-5 * 0.2**epoch

In [54]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    
    x1 = tf.keras.layers.Dropout(0.15)(x[0]) 
    x1 = tf.keras.layers.Conv1D(128, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(64, 2,padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.15)(x[0]) 
    x2 = tf.keras.layers.Conv1D(128, 2, padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(64, 2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return model

## Train

In [55]:
# checkup
print(train_df.shape, test_df.shape)

(21984, 4) (5496, 3)


In [56]:
n_splits = 5

In [72]:
"""
jac = []; VER='v3'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=777)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train_df.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
        
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

    sv = tf.keras.callbacks.ModelCheckpoint(
        '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')
        
    hist = model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
        epochs=5, batch_size=8, verbose=DISPLAY, callbacks=[sv, reduce_lr],
        validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
        [start_tokens[idxV,], end_tokens[idxV,]]))
    
    print('Loading model...')
    model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    
    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train_df.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train_df.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-1:b])
        all.append(jaccard(st,train_df.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()
"""

#########################
### FOLD 1
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0336 - activation_loss: 0.0170 - activation_1_loss: 0.0166
Epoch 00001: val_loss improved from inf to 0.02882, saving model to v3-roberta-0.h5
2199/2199 [==============================] - 340s 155ms/step - loss: 0.0336 - activation_loss: 0.0170 - activation_1_loss: 0.0166 - val_loss: 0.0288 - val_activation_loss: 0.0150 - val_activation_1_loss: 0.0139 - lr: 3.0000e-05
Epoch 2/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0260 - activation_loss: 0.0133 - activation_1_loss: 0.0127
Epoch 00002: val_loss improved from 0.02882 to 0.02727, saving model to v3-roberta-0.h5
2199/2199 [==============================] - 330s 150ms/step - loss: 0.0260 - activation_loss: 0.0133 - activation_1_loss: 0.0127 - val_loss: 0.0273 - val_activation_loss: 0.0139 - val_activation_1_loss: 0.0134 - lr: 6.0000e-06
Epoch 3/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0241 - activation_loss: 0.0124 - ac

All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0333 - activation_loss: 0.0169 - activation_1_loss: 0.0164
Epoch 00001: val_loss improved from inf to 0.02895, saving model to v3-roberta-1.h5
2199/2199 [==============================] - 329s 150ms/step - loss: 0.0333 - activation_loss: 0.0169 - activation_1_loss: 0.0164 - val_loss: 0.0289 - val_activation_loss: 0.0148 - val_activation_1_loss: 0.0141 - lr: 3.0000e-05
Epoch 2/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0257 - activation_loss: 0.0131 - activation_1_loss: 0.0126
Epoch 00002: val_loss improved from 0.02895 to 0.02671, saving model to v3-roberta-1.h5
2199/2199 [==============================] - 323s 147ms/step - loss: 0.0257 - activation_loss: 0.0131 - activation_1_loss: 0.0126 - val_loss: 0.0267 - val_activation_loss: 0.0137 - val_activation_1_loss: 0.0130 - lr: 6.0000e-06
Epoch 3/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0238 - activation_loss: 0.0121 - ac

...

All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0335 - activation_loss: 0.0169 - activation_1_loss: 0.0165
Epoch 00001: val_loss improved from inf to 0.02849, saving model to v3-roberta-3.h5
2199/2199 [==============================] - 336s 153ms/step - loss: 0.0335 - activation_loss: 0.0169 - activation_1_loss: 0.0165 - val_loss: 0.0285 - val_activation_loss: 0.0145 - val_activation_1_loss: 0.0140 - lr: 3.0000e-05
Epoch 2/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0261 - activation_loss: 0.0134 - activation_1_loss: 0.0127
Epoch 00002: val_loss improved from 0.02849 to 0.02743, saving model to v3-roberta-3.h5
2199/2199 [==============================] - 333s 152ms/step - loss: 0.0261 - activation_loss: 0.0134 - activation_1_loss: 0.0127 - val_loss: 0.0274 - val_activation_loss: 0.0139 - val_activation_1_loss: 0.0136 - lr: 6.0000e-06
Epoch 3/5
2199/2199 [==============================] - ETA: 0s - loss: 0.0242 - activation_loss: 0.0125 - ac

All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/5
   6/2199 [..............................] - ETA: 4:15 - loss: 0.1103 - activation_loss: 0.0571 - activation_1_loss: 0.0532

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[8,96,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/dropout_37/dropout/Mul_1 (defined at <ipython-input-72-b2c0a9e8dc6f>:24) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[gradient_tape/model/tf_roberta_model/roberta/embeddings/position_embeddings/embedding_lookup/Reshape/_576]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[8,96,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/dropout_37/dropout/Mul_1 (defined at <ipython-input-72-b2c0a9e8dc6f>:24) ]]
Hint: If you want to see a li

In [60]:
tf.test.is_gpu_available()

True

In [61]:
## Inference

In [62]:
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))
DISPLAY=1
model = ""
for i in range(4):
    print('#'*25)
    print('### MODEL %i'%(i+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
    model.load_weights('./v3-roberta-%i.h5'%i)

    print('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/n_splits
    preds_end += preds[1]/n_splits

#########################
### MODEL 1
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Predicting Test...
172/172 [==============================] - 27s 158ms/step
#########################
### MODEL 2
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Predicting Test...
172/172 [==============================] - 26s 153ms/step
#########################
### MODEL 3
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Predicting Test...
172/172 [==============================] - 27s 154ms/step
#########################
### MODEL 4
#########################


All model checkpoint weights were used when initializing TFRobertaModel.

All the weights of TFRobertaModel were initialized from the model checkpoint at ./input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Predicting Test...
172/172 [==============================] - 27s 157ms/step


In [63]:
# import io
# stream = io.StringIO()
# model.summary(print_fn=lambda x: stream.write(x))
# summary_string = stream.getvalue()
# stream.close()
# summary_string

In [64]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 96)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 96)]         0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 96, 768), (N 124645632   input_1[0][0]                    
______________________________________________________________________________________________

In [65]:
tf.keras.utils.plot_model(model, to_file='roberta_v3.png', show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [66]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test_df.loc[k,'text']
    else:
        text1 = " "+" ".join(test_df.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-1:b])
    all.append(st)

In [67]:
test_df['selected_text'] = all
test_df[['textID','selected_text']].to_csv('submission_v3.csv',index=False)

In [68]:
test_inf = pd.read_csv("./submission_v3.csv")
test_inf.shape

(5496, 2)

In [69]:
test_inf.head(10)

,textID,selected_text
0,85a02242e3,"its been a slow day at home, one of my kids i..."
1,a0e0ed4311,missed
2,f5f2a709ca,i like them.
3,064985c587,grad present photos on facebook
4,5ab37ce023,"that`s kinda cute,"
5,4d8f747e25,yay!
6,1bb6bb79eb,"exhausted,"
7,06e3f466dc,"right, i must be off to do some sewing. bye x"
8,20eb1910eb,can`t imagine a better long weekend
9,f9f99c41a6,ugh talk to me someone i`m really bored.


In [70]:
answer_st[:10]

0    Its been a slow day at home, one of my kids is...
1                                               missed
2                                         I like them.
3                      Grad present photos on facebook
4                             kinda cute, to be honest
5                                                 yay!
6                                           exhausted,
7        Right, I must be off to do some sewing. Bye x
8                  Can`t imagine a better long weekend
9                                               bored.
Name: selected_text, dtype: object

In [73]:
scores = []
for i in range(test_inf.shape[0]):
    scores.append(jaccard(test_inf["selected_text"][i], answer_st[i]))
print("average jaccard score:", sum(scores) / len(scores))

average jaccard score: 0.7109999153175082


In [75]:
len(scores)

5496

In [76]:
test.shape

(5496, 4)

In [79]:
test.columns.tolist()

['textID', 'text', 'selected_text', 'sentiment']

In [138]:
answer_st[1].lower() == test_inf["selected_text"][1]

False

In [143]:
cnt = test.shape[0]
mismatch = []

for i in range(cnt):
    if scores[i] < 0.1:
        mismatch.append([answer_st[i].lower(), test_inf["selected_text"][i]])

mismatch = pd.DataFrame(mismatch, columns=["y", "y_hat"])

In [144]:
print(mismatch.shape)
mismatch.head(30)

(502, 2)


,y,y_hat
0,i only have $4.38 in my account,i certainly know the feeling.
1,may the forth be with you... ha yes... today i...,lucky
2,good to be back & go to the junction with the ...,good
3,awww i`ll let you off - but you`d better be t...,awww
4,failed on first day. twic,not going well.
5,i liked,DEMI YOU WERE SO PRETTY WITH YOU`RE BANGS AND...
6,hopefully,good
7,enjoyed,i hope u enjoyed it and your back is now feel...
8,love,i love you but i can`t call you cause i live ...
9,i now have brian freeze,wow i just drank a drink of water - 12 ice cu...


- eda 설명추가
- 